<a href="https://colab.research.google.com/github/tbrandonstevenson/Data-Exercises/blob/main/RevenueClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>